In [1]:
import requests
import qdrant_client
import importlib.metadata
import langchain
import getpass
import os

print(langchain.__version__)
print(requests.__version__)
print(importlib.metadata.version("qdrant-client"))
print(importlib.metadata.version("langchain_openai"))
# print(qdrant_client.__version__)

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

0.3.19
2.32.3
1.13.2
0.3.6


In [4]:
AMADEUS_CLIENT_ID = "1nsyVeNCT6PQtpqb1iG5MsIPWTkvN8bU"
AMADEUS_CLIENT_SECRET = "GizAt9lxJi00tWG7"

In [5]:
AMADEUS_AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"

def get_amadeus_access_token():
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_CLIENT_ID,
        "client_secret": AMADEUS_CLIENT_SECRET
    }
    
    response = requests.post(AMADEUS_AUTH_URL, headers=headers, data=data)

    if response.status_code == 200:
        token = response.json().get("access_token")
        print("✅ Access token received:", token)  # Print token for debugging
        return token
    else:
        print("❌ Failed to get access token:", response.json())
        return None

# Get and print the access token
access_token = get_amadeus_access_token()

✅ Access token received: bcF8lTGwK0hJm7tv2I3AoUA4I7Su


In [6]:
if access_token:
    FLIGHTS_API_URL = "https://test.api.amadeus.com/v2/shopping/flight-offers"

    headers = {"Authorization": f"Bearer {access_token}"}

    params = {
        "originLocationCode": "NYC",
        "destinationLocationCode": "LON",
        "departureDate": "2025-05-01",
        "adults": 1,
        "max": 3
    }

    response = requests.get(FLIGHTS_API_URL, headers=headers, params=params)

    if response.status_code == 200:
        print("✅ API response received:", response.json())
    else:
        print("❌ API Error:", response.json())


✅ API response received: {'meta': {'count': 3, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=NYC&destinationLocationCode=LON&departureDate=2025-05-01&adults=1&max=3'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-02-21', 'lastTicketingDateTime': '2025-02-21', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT7H25M', 'segments': [{'departure': {'iataCode': 'JFK', 'terminal': '4', 'at': '2025-05-01T18:00:00'}, 'arrival': {'iataCode': 'LHR', 'terminal': '3', 'at': '2025-05-02T06:25:00'}, 'carrierCode': 'VS', 'number': '4', 'aircraft': {'code': '339'}, 'operating': {'carrierCode': 'VS'}, 'duration': 'PT7H25M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '301.18', 'base': '116.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amoun

In [2]:
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from langchain_openai import OpenAIEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
import json

In [9]:
# Initialize Qdrant
qdrant_client = QdrantClient(":memory:")  # Use persistent URL for real use case

In [10]:
# Define collection
qdrant_client.create_collection(
    collection_name="travel_data_002",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)  # Ensure size matches OpenAI embeddings
)

True

In [3]:
AMADEUS_CLIENT_ID = "1nsyVeNCT6PQtpqb1iG5MsIPWTkvN8bU"
AMADEUS_CLIENT_SECRET = "GizAt9lxJi00tWG7"
AMADEUS_AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"

# Step 1: Get Access Token
def get_amadeus_access_token():
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "grant_type": "client_credentials",
        "client_id": AMADEUS_CLIENT_ID,
        "client_secret": AMADEUS_CLIENT_SECRET
    }
    
    response = requests.post(AMADEUS_AUTH_URL, headers=headers, data=data)

    if response.status_code == 200:
        token = response.json().get("access_token")
        print("✅ Access token received:", token)  # Print token for debugging
        return token
    else:
        print("❌ Failed to get access token:", response.json())
        return None

# Get and print the access token
access_token = get_amadeus_access_token()

✅ Access token received: zMWvXwKXJYYKM9mqbHeHwmhcAxji


In [4]:
if access_token:
    FLIGHTS_API_URL = "https://test.api.amadeus.com/v2/shopping/flight-offers"

    headers = {"Authorization": f"Bearer {access_token}"}

    params = {
        "originLocationCode": "NYC",
        "destinationLocationCode": "LON",
        "departureDate": "2025-05-01",
        "adults": 1,
        "max": 3
    }

    response = requests.get(FLIGHTS_API_URL, headers=headers, params=params)

    if response.status_code == 200:
        print("✅ API response received:", response.json())
    else:
        print("❌ API Error:", response.json())

✅ API response received: {'meta': {'count': 3, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=NYC&destinationLocationCode=LON&departureDate=2025-05-01&adults=1&max=3'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-02-21', 'lastTicketingDateTime': '2025-02-21', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT7H25M', 'segments': [{'departure': {'iataCode': 'JFK', 'terminal': '4', 'at': '2025-05-01T18:00:00'}, 'arrival': {'iataCode': 'LHR', 'terminal': '3', 'at': '2025-05-02T06:25:00'}, 'carrierCode': 'VS', 'number': '4', 'aircraft': {'code': '339'}, 'operating': {'carrierCode': 'VS'}, 'duration': 'PT7H25M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '301.18', 'base': '116.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amoun

In [9]:
# Step 2: Fetch Flight Offers from Amadeus
def fetch_flight_data(origin, destination, departure_date):
    access_token = get_amadeus_access_token()
    FLIGHTS_API_URL = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    # url = f"https://test.api.amadeus.com/v2/shopping/flight-offers?origin={origin}&destination={destination}&departureDate={departure_date}&nonStop=false&max=3"

    headers = {"Authorization": f"Bearer {access_token}"}
    
    params = {
        "originLocationCode": origin,
        "destinationLocationCode": destination,
        "departureDate": departure_date,
        "adults": 1,
        "max": 3
    }
    
    response = requests.get(FLIGHTS_API_URL, headers=headers, params=params)

    if response.status_code == 200:
        print("✅ API response received:", response.json())
    else:
        print("❌ API Error:", response.json())    

In [10]:
tmp = fetch_flight_data("NYC", "LON", "2025-05-01")

✅ Access token received: GIsmGWeqICZPc0LG4uL56nRcAeUz
✅ API response received: {'meta': {'count': 3, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=NYC&destinationLocationCode=LON&departureDate=2025-05-01&adults=1&max=3'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-02-21', 'lastTicketingDateTime': '2025-02-21', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT7H25M', 'segments': [{'departure': {'iataCode': 'JFK', 'terminal': '4', 'at': '2025-05-01T18:00:00'}, 'arrival': {'iataCode': 'LHR', 'terminal': '3', 'at': '2025-05-02T06:25:00'}, 'carrierCode': 'VS', 'number': '4', 'aircraft': {'code': '339'}, 'operating': {'carrierCode': 'VS'}, 'duration': 'PT7H25M', 'id': '1', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '301.18', 'base': '116.00', 'f

In [11]:
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Step 1: Process and Embed Data
def preprocess_and_store(data):
    all_flights = data.get("data", [])

    points = []
    for index, flight in enumerate(all_flights):
        text_data = json.dumps(flight, indent=2)  # Convert JSON to text
        vector = embeddings.embed_query(text_data)  # Get vector embedding

        # Store in Qdrant
        points.append(
            PointStruct(id=index, vector=vector, payload={"content": text_data})
        )

    # Insert into Qdrant
    qdrant_client.upsert(collection_name="travel_data_002", points=points)

In [12]:
# Fetch sample flight data
flight_data = fetch_flight_data("NYC", "LON", "2025-05-01")
print(flight_data)

✅ Access token received: ABjx4Cx6jhlSAgEbAB6ljWnNGzJe
✅ API response received: {'meta': {'count': 3, 'links': {'self': 'https://test.api.amadeus.com/v2/shopping/flight-offers?originLocationCode=NYC&destinationLocationCode=LON&departureDate=2025-05-01&adults=1&max=3'}}, 'data': [{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'isUpsellOffer': False, 'lastTicketingDate': '2025-02-21', 'lastTicketingDateTime': '2025-02-21', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT7H25M', 'segments': [{'departure': {'iataCode': 'JFK', 'terminal': '4', 'at': '2025-05-01T18:00:00'}, 'arrival': {'iataCode': 'LHR', 'terminal': '3', 'at': '2025-05-02T06:25:00'}, 'carrierCode': 'VS', 'number': '4', 'aircraft': {'code': '339'}, 'operating': {'carrierCode': 'VS'}, 'duration': 'PT7H25M', 'id': '3', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '301.18', 'base': '116.00', 'f

In [13]:
# Process and store Amadeus data in Qdrant
preprocess_and_store(flight_data)

AttributeError: 'NoneType' object has no attribute 'get'